In [199]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math

In [200]:
df['summary'].isna().sum()


0

In [201]:
df=pd.read_csv("/Users/swastikagarwal/Downloads/SEM5_RVCE/LAB WORK/NLP/nlp dataset/Musical_instruments_reviews 4.csv")
df.dropna(subset=['reviewText'], inplace=True)
X = df['summary']
y = df.iloc[:, 5]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)
df


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5.0,Five Stars,1405814400,"07 20, 2014"
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",I've been thinking about trying the Nanoweb st...,5.0,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014"
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",I have tried coated strings in the past ( incl...,4.0,Good for coated.,1405987200,"07 22, 2014"
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4.0,Taylor Made,1404172800,"07 1, 2014"


In [202]:
def preprocess(text):
    stop_words=set(stopwords.words('english'))
    # text=text.lower()
    words=word_tokenize(text)
    list1=[]

    for word in words:
        if word not in stop_words and word.isalnum():
            list1.append(word)
    
    list_doc=' '.join(list1)
    return list_doc

In [203]:
X_train=X_train.apply(preprocess)
X_test=X_test.apply(preprocess)
# X_processed = X.apply(preprocess)


In [204]:
def dff(word,sentence):
    words=sentence.split()
    return words.count(word)/(len(words)+1)

def idff(word,sentences):
    n=0
    for sent in sentences:
        if word in sent:
            n+=1
    return math.log(len(sentences)/(n+1))

all_docs=X_train.to_list()+X_test.to_list()

unique_words=set(' '.join(all_docs).split())

idf_score={}
for word in unique_words:
    idf_score[word]=idff(word,all_docs)

#vocabulary
vocab=list(idf_score.keys())

X_train_vector=[]
for sentence in X_train:
    temp_list=[]
    for w in vocab:
        tfidf_score=dff(w,sentence)*idf_score[w]
        temp_list.append(tfidf_score)
    X_train_vector.append(temp_list)


X_test_vector = []
for sentence in X_test:
    temp_list = []
    for w in vocab:
        tfidf_score = dff(w, sentence)*idf_score[w]
        temp_list.append(tfidf_score)
    X_test_vector.append(temp_list)

In [205]:
X_train_vector=np.array(X_train_vector)
X_test_vector=np.array(X_test_vector)

In [206]:
model=LogisticRegression(max_iter=100)
model.fit(X_train_vector,y_train)

/Users/swastikagarwal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [211]:
text = "serious musicians look elsewhere"

text=preprocess(text)
vector=[]

for word in vocab:
    tfs=dff(word,text)
    tf_idf_score=tfs*idf_score[word]
    vector.append(tf_idf_score)

output=model.predict([vector])
output


array([3.])